In [1]:
from PPOAgent import PPOAgent
from SubEnvironment import SubEnvironment
import time
%load_ext autoreload
%autoreload 2

In [2]:
env = SubEnvironment(max_steps=512, period=128, lookback=64,
                     frequency=(1,10), freq_var=(0,1.),
                     amplitude=1., amp_var=(0., 0.5),
                     noise=0.1, phase=(0,1))

In [69]:
agent = PPOAgent(env, verbose=1)
agent.run(2)

2019-01-08 22:03:17,411 Run Created: ./runs/agent-34/
2019-01-08 22:03:17,436 Agen logs created.
2019-01-08 22:03:17,440 Running Pretraining Step.
2019-01-08 22:03:17,443 Building Models.
2019-01-08 22:03:17,720 Beginning Training Loop.
2019-01-08 22:03:18,423 Episode 0 done! Reward: -46.74787176926317
2019-01-08 22:03:20,915 Episode 1 done! Reward: -26.086009661134995
2019-01-08 22:03:21,153 Training Complete!


In [74]:
import gym
env = gym.make('CartPole-v0')
agent = PPOAgent(env, verbose=1)
agent.run(100)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


2019-01-08 22:06:40,654 Run Created: ./runs/agent-37/
2019-01-08 22:06:40,657 Agen logs created.
2019-01-08 22:06:40,658 Running Pretraining Step.
2019-01-08 22:06:40,660 Building Models.
2019-01-08 22:06:40,931 Beginning Training Loop.
2019-01-08 22:06:41,073 Episode 0 done! Reward: 15.0
2019-01-08 22:06:41,098 Episode 1 done! Reward: 17.0
2019-01-08 22:06:41,137 Episode 2 done! Reward: 27.0
2019-01-08 22:06:41,168 Episode 3 done! Reward: 21.0
2019-01-08 22:06:41,201 Episode 4 done! Reward: 24.0
2019-01-08 22:06:41,326 Episode 5 done! Reward: 88.0
2019-01-08 22:06:41,350 Episode 6 done! Reward: 16.0
2019-01-08 22:06:41,382 Episode 7 done! Reward: 22.0
2019-01-08 22:06:41,411 Episode 8 done! Reward: 18.0
2019-01-08 22:06:41,450 Episode 9 done! Reward: 30.0
2019-01-08 22:06:43,383 Episode 10 done! Reward: 23.0
2019-01-08 22:06:43,426 Episode 11 done! Reward: 26.0
2019-01-08 22:06:43,457 Episode 12 done! Reward: 18.0
2019-01-08 22:06:43,521 Episode 13 done! Reward: 45.0
2019-01-08 22:06:

In [33]:
import threading
from IPython.display import display
import ipywidgets as widgets
from sidecar import Sidecar

In [34]:
run_dir = "./runs/agent-27/data/"
line_limit = 10
log_path = run_dir + 'agent.txt'
log_val = []
kill_thread = False
last_frame_count = 0
empty_frame_count = 0

def update_log():
    log_val = ['<li><code style="font-size: 10px;">{}</code></li>'.format(line) for 
               line in list(open(log_path))[-line_limit:]]
    log_widget.value = '<ul style="list-style-type: none; padding: 0;">' + ''.join(log_val) + '</ul>'
    
def on_change(x):
    if (x['name'] == 'value'):
        update_image(x['new'])
        
def kill_thread_click(x):
    global kill_thread
    kill_thread = True
        
def live_update():
    global last_frame_count
    global empty_frame_count
    while True:
        update_count_widget.value = "Last Update: " + time.strftime('%c')
        update_log()
        if last_frame_count == frame_count:
            empty_frame_count += 1
        else:
            last_frame_count = frame_count
            empty_frame_count = 0
        if empty_frame_count > 10 or kill_thread:
            update_count_widget.value = 'Halted: ' + time.strftime('%c')
            break
        time.sleep(1)

play_widget = widgets.Play(
    interval=250,
    value=frame_count,
    min=0,
    max=frame_count,
    step=1,
    description="Press play",
    disabled=False
)
slider_widget = widgets.IntSlider(max=frame_count)
image_widget = widgets.Image(format='png', height=100)
 
slider_widget.observe(on_change)
widgets.jslink((play_widget, 'value'), (slider_widget, 'value'))

log_widget = widgets.HTML()

update_count_widget = widgets.Label()

kill_button_widget = widgets.Button(
    description='Kill Thread'
)
kill_button_widget.on_click(kill_thread_click)

thread = threading.Thread(target=live_update)
thread.start()

training_info = widgets.VBox([image_widget,
                      widgets.HBox([kill_button_widget, play_widget, slider_widget, update_count_widget]),
                      log_widget])

sc = Sidecar(title='Training Info')
with sc:
    display(training_info)

FileNotFoundError: [Errno 2] No such file or directory: './runs/agent-27/data/animations/meta_plot/'